# 1: 位置 $x$ と時刻 $t$ における波の振幅 $a(x, t)$ の式

$$
a(x, t) = A \sin(kx - \omega t + \phi) \hspace{30pt} (1) 
$$

ここで $A$ は振幅、$k$ は波数、$\omega$ は振動数、$\phi$ は位相を表します。

以下のコードでは波数、振動数、位相を変化させたときの波の形をアニメーションで描画します。

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
from matplotlib import cm
from IPython.display import HTML
import matplotlib

# アニメーションの埋め込み制限を増やす
matplotlib.rcParams['animation.embed_limit'] = 100  # MB

# 時空間での波の伝播可視化
# Y軸を時間軸として使用

# ハードコードされたパラメータ
A = 1.0                    # 振幅
k = 1.0                    # 波数
omega = 2.0                # 角周波数
phi = 0.0                  # 初期位相

# 導出される値
wavelength = 2 * np.pi / k
period = 2 * np.pi / omega
phase_velocity = omega / k

print(f"Wave parameters:")
print(f"  Wave number: k = {k:.2f} rad/m")
print(f"  Angular frequency: ω = {omega:.2f} rad/s")
print(f"  Wavelength: λ = {wavelength:.2f} m")
print(f"  Period: T = {period:.2f} s")
print(f"  Phase velocity: c = {phase_velocity:.2f} m/s")

# 空間と時間のグリッド
x = np.linspace(-5, 10, 200)
t = np.linspace(0, 10, 200)
X, T = np.meshgrid(x, t)

# 波動場の計算
# a(x,t) = A * sin(kx - ωt + φ)
Z = A * np.sin(k*X - omega*T + phi)

# 監視する2点
x1, x2 = 0, 2
idx1 = np.argmin(np.abs(x - x1))
idx2 = np.argmin(np.abs(x - x2))

# アニメーション版
fig_anim = plt.figure(figsize=(16, 10))

# アニメーション用のプロット
ax_contour = fig_anim.add_subplot(1, 2, 1)
ax_time = fig_anim.add_subplot(1, 2, 2)

# 固定された時間範囲
t_start = 0
t_end = 10
dt = 0.05
n_frames = int(t_end / dt)

# 固定されたグリッド（アニメーション全体で共通）
t_fixed = np.linspace(t_start, t_end, 200)
x_fixed = np.linspace(-5, 10, 200)
X_fixed, T_fixed = np.meshgrid(x_fixed, t_fixed)
Z_fixed = A * np.sin(k*X_fixed - omega*T_fixed + phi)

def init_anim():
    ax_contour.clear()
    ax_time.clear()
    return []

def animate(frame):
    ax_contour.clear()
    ax_time.clear()
    
    current_time = frame * dt
    
    # カラーコントアプロット（固定されたデータを使用）
    contour = ax_contour.contourf(X_fixed, T_fixed, Z_fixed, 
                                  levels=np.linspace(-1, 1, 21),
                                  cmap=cm.coolwarm, extend='both')
    
    # 等高線を追加
    ax_contour.contour(X_fixed, T_fixed, Z_fixed, 
                       levels=np.linspace(-1, 1, 11),
                       colors='black', alpha=0.3, linewidths=0.5)
    
    # 現在時刻を示す線（これだけが動く）
    ax_contour.axhline(y=current_time, color='green', linewidth=3, linestyle='-', label='Current time')
    
    # 監視点の位置を示す線
    ax_contour.axvline(x=x1, color='red', linewidth=1, linestyle=':', alpha=0.7)
    ax_contour.axvline(x=x2, color='blue', linewidth=1, linestyle=':', alpha=0.7)
    
    # 監視点での軌跡を強調
    ax_contour.plot(np.full_like(t_fixed, x1), t_fixed, 'w-', linewidth=2, alpha=0.5)
    ax_contour.plot(np.full_like(t_fixed, x2), t_fixed, 'w-', linewidth=2, alpha=0.5)
    
    ax_contour.set_xlabel('Position x [m]', fontsize=12)
    ax_contour.set_ylabel('Time t [s]', fontsize=12)
    ax_contour.set_title(f'Space-Time Wave Field (Contour Plot)', fontsize=14)
    ax_contour.set_xlim(-5, 10)
    ax_contour.set_ylim(t_start, t_end)
    
    # カラーバーを追加（初回のみ）
    if frame == 0:
        cbar = plt.colorbar(contour, ax=ax_contour)
        cbar.set_label('Amplitude', fontsize=12)
    
    # 右側のパネル：現在時刻での空間分布
    current_wave = A * np.sin(k*x - omega*current_time + phi)
    ax_time.plot(x, current_wave, 'k-', linewidth=2)
    
    # 監視点をマーク
    ax_time.plot(x1, A * np.sin(k*x1 - omega*current_time + phi), 'ro', markersize=10, label=f'x = {x1}')
    ax_time.plot(x2, A * np.sin(k*x2 - omega*current_time + phi), 'bo', markersize=10, label=f'x = {x2}')
    
    ax_time.set_xlabel('Position x [m]', fontsize=12)
    ax_time.set_ylabel('Amplitude', fontsize=12)
    ax_time.set_title(f'Spatial Distribution at t = {current_time:.2f} s', fontsize=14)
    ax_time.set_xlim(-5, 10)
    ax_time.set_ylim(-1.5, 1.5)
    ax_time.grid(True, alpha=0.3)
    ax_time.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    ax_time.axvline(x=x1, color='red', linestyle=':', alpha=0.5)
    ax_time.axvline(x=x2, color='blue', linestyle=':', alpha=0.5)
    ax_time.legend()
    
    # 波の情報を表示
    phase_at_x1 = (k*x1 - omega*current_time + phi) % (2*np.pi)
    phase_at_x2 = (k*x2 - omega*current_time + phi) % (2*np.pi)
    info_text = f'Phase at x={x1}: {phase_at_x1:.2f} rad\nPhase at x={x2}: {phase_at_x2:.2f} rad'
    ax_time.text(0.05, 0.95, info_text, transform=ax_time.transAxes, 
                verticalalignment='top', fontsize=10,
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    return []

# アニメーションの作成
anim = animation.FuncAnimation(fig_anim, animate, init_func=init_anim,
                              frames=n_frames, interval=50,
                              blit=False, repeat=True)

# HTMLとして表示
HTML(anim.to_jshtml())

# 2: F-K変換

F-K変換は、波の振幅を空間と時間の関数として表現する方法です。波の振幅 $a(x, t)$ をフーリエ変換すると、以下のように表されます。

$$
\begin{align*}
A(\omega_i, x) &= \frac{1}{N} \sum_{n=0}^{N-1} a(x, t_n) e^{-i \omega_i t_n} \hspace{30pt} (2-3) \\
A(k_i, \omega_i) &= \frac{1}{M} \sum_{m=0}^{M-1} A(\omega_i, x_m) e^{-i k_i x_m} \hspace{30pt} (2-4)
\end{align*}
$$

以下のコードでは、F-K変換を用いて波の振幅を空間と時間の関数として表現し、アニメーションで表示します。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.colors as colors

# F-K変換の可視化
# (x,t)空間から(k,ω)空間への変換を理解する

# パラメータ設定
# 位相速度の差が明瞭になるように設定
wave_params = [
    {'k': 10.0, 'omega': 3.0, 'A': 1.0, 'phi': 0.0},      # 波1: c = 0.3 m/s (低速)
    {'k': 5.0, 'omega': 5.0, 'A': 0.8, 'phi': np.pi/4},   # 波2: c = 1.0 m/s (中速)
    {'k': 4.0, 'omega': 20.0, 'A': 0.6, 'phi': -np.pi/3}, # 波3: c = 5.0 m/s (高速)
]

# 各波の位相速度を計算
for params in wave_params:
    params['c'] = params['omega'] / params['k']

# 空間と時間のグリッド
nx, nt = 256, 256
x_max, t_max = 10, 10
x = np.linspace(0, x_max, nx)
t = np.linspace(0, t_max, nt)
dx = x[1] - x[0]
dt = t[1] - t[0]

# 波動場の生成（複数の波の重ね合わせ）
X, T = np.meshgrid(x, t)
wave_field = np.zeros_like(X)

print("波のパラメータ:")
for i, params in enumerate(wave_params):
    wave_i = params['A'] * np.sin(params['k']*X - params['omega']*T + params['phi'])
    wave_field += wave_i
    print(f"  波{i+1}: k={params['k']:.1f} rad/m, ω={params['omega']:.1f} rad/s, A={params['A']:.1f}, c={params['c']:.2f} m/s")

# F-K変換の実行
# Step 1: 時間方向のFFT（各位置で）
fft_t = np.fft.fft(wave_field, axis=0)
omega_array = 2 * np.pi * np.fft.fftfreq(nt, dt)

# Step 2: 空間方向のFFT（各周波数で）
fft_xt = np.fft.fft(fft_t, axis=1)
k_array = 2 * np.pi * np.fft.fftfreq(nx, dx)

# F-Kスペクトルの計算（パワー）
fk_spectrum = np.abs(fft_xt)**2

# 正の周波数・波数領域のみ抽出
omega_positive = omega_array[:nt//2]
k_positive = k_array[:nx//2]
fk_spectrum_positive = fk_spectrum[:nt//2, :nx//2]

# 静的な可視化
fig = plt.figure(figsize=(16, 10))

# 断面の位置を定義
x_section = x_max / 4  # x = 2.5 m
omega_section = 5.0    # ω = 5.0 rad/s
x_idx = np.argmin(np.abs(x - x_section))
omega_idx = np.argmin(np.abs(omega_positive - omega_section))

# 1. 時空間波動場（時間軸を下から上に）
ax1 = fig.add_subplot(2, 3, 1)
im1 = ax1.imshow(wave_field, extent=[0, x_max, 0, t_max], 
                 cmap='coolwarm', aspect='auto', vmin=-2, vmax=2, origin='lower')
ax1.set_xlabel('Position x [m]')
ax1.set_ylabel('Time t [s]')
ax1.set_title('Wave Field in (x,t) Space')
plt.colorbar(im1, ax=ax1, label='Amplitude')

# 理論的な波の軌跡を追加
for params in wave_params:
    t_line = np.linspace(0, t_max, 100)
    x_line = params['c'] * t_line
    mask = x_line <= x_max
    ax1.plot(x_line[mask], t_line[mask], '--', linewidth=1, alpha=0.7)

# 断面線を追加（ax4とax5の位置）
ax1.axvline(x=x_section, color='green', linestyle=':', linewidth=2, label=f'x = {x_section:.1f} m')
ax1.legend()

# 2. F-Kスペクトル（2D）
ax2 = fig.add_subplot(2, 3, 2)
# log10スケールでのパワー
log_power = np.log10(fk_spectrum_positive + 1e-10)
im2 = ax2.imshow(log_power, 
                 extent=[0, k_positive.max(), 0, omega_positive.max()],
                 cmap='viridis', aspect='auto', origin='lower')
ax2.set_xlabel('Wave number k [rad/m]')
ax2.set_ylabel('Angular frequency ω [rad/s]')
ax2.set_title('F-K Spectrum (log scale)')
ax2.set_xlim(0, 15)
ax2.set_ylim(0, 25)
plt.colorbar(im2, ax=ax2, label='log10(Power)')

# 理論的な分散関係を追加
k_theory = np.linspace(0, 15, 100)
for params in wave_params:
    omega_theory = params['c'] * k_theory
    ax2.plot(k_theory, omega_theory, 'w--', linewidth=1, alpha=0.7)

# 断面線を追加（ax3の位置）
ax2.axhline(y=omega_section, color='red', linestyle=':', linewidth=2, label=f'ω = {omega_section:.1f} rad/s')
ax2.legend()

# 3. 特定周波数での波数スペクトル
ax3 = fig.add_subplot(2, 3, 3)
k_spectrum_at_omega = fk_spectrum_positive[omega_idx, :]
ax3.plot(k_positive, k_spectrum_at_omega)
ax3.set_xlabel('Wave number k [rad/m]')
ax3.set_ylabel('Power')
ax3.set_title(f'Wave Number Spectrum at ω = {omega_positive[omega_idx]:.2f} rad/s')
ax3.set_xlim(0, 15)
ax3.grid(True, alpha=0.3)

# ピークの位置をマーク
peaks_idx = np.where(k_spectrum_at_omega > np.max(k_spectrum_at_omega) * 0.1)[0]
for idx in peaks_idx:
    if k_positive[idx] > 0:
        c_est = omega_positive[omega_idx] / k_positive[idx]
        ax3.axvline(x=k_positive[idx], color='red', linestyle=':', alpha=0.5)
        ax3.text(k_positive[idx], np.max(k_spectrum_at_omega)*0.8, 
                f'c≈{c_est:.1f}', ha='center', fontsize=9)

# 4. 時間領域の信号（特定位置）
ax4 = fig.add_subplot(2, 3, 4)
time_signal = wave_field[:, x_idx]
ax4.plot(t, time_signal)
ax4.set_xlabel('Time t [s]')
ax4.set_ylabel('Amplitude')
ax4.set_title(f'Time Signal at x = {x[x_idx]:.1f} m')
ax4.grid(True, alpha=0.3)

# 5. 周波数スペクトル（特定位置）
ax5 = fig.add_subplot(2, 3, 5)
freq_spectrum = np.abs(fft_t[:nt//2, x_idx])
ax5.plot(omega_positive, freq_spectrum)
ax5.set_xlabel('Angular frequency ω [rad/s]')
ax5.set_ylabel('Amplitude')
ax5.set_title(f'Frequency Spectrum at x = {x[x_idx]:.1f} m')
ax5.set_xlim(0, 25)
ax5.grid(True, alpha=0.3)

# 主要な周波数成分をマーク
peaks_omega = omega_positive[freq_spectrum > np.max(freq_spectrum) * 0.2]
for peak in peaks_omega:
    ax5.axvline(x=peak, color='red', linestyle=':', alpha=0.5)

# 6. 位相速度の抽出（マーカーのサイズをlog10(Power)に比例）
ax6 = fig.add_subplot(2, 3, 6)

# F-Kスペクトルのピークから位相速度を推定
c_values = []
omega_values = []
power_values = []

for i in range(len(omega_positive)):
    if omega_positive[i] > 1.0:  # 低周波ノイズを除外
        k_peak_idx = np.argmax(fk_spectrum_positive[i, :])
        if k_positive[k_peak_idx] > 0.5:  # 低波数ノイズを除外
            c_estimated = omega_positive[i] / k_positive[k_peak_idx]
            if c_estimated < 10:  # 異常値を除外
                c_values.append(c_estimated)
                omega_values.append(omega_positive[i])
                power_values.append(fk_spectrum_positive[i, k_peak_idx])

# log10(Power)を計算してマーカーサイズに変換
log_power_values = np.log10(np.array(power_values) + 1e-10)
# 正規化してサイズを決定（5から200の範囲）
size_min, size_max = 5, 200
log_power_normalized = (log_power_values - np.min(log_power_values)) / (np.max(log_power_values) - np.min(log_power_values))
marker_sizes = size_min + (size_max - size_min) * log_power_normalized

# 散布図（サイズをパワーに対応）
scatter = ax6.scatter(omega_values, c_values, s=marker_sizes, 
                     c='blue', alpha=0.6, edgecolors='darkblue', linewidth=0.5)

ax6.set_xlabel('Angular frequency ω [rad/s]')
ax6.set_ylabel('Phase velocity c [m/s]')
ax6.set_title('Extracted Phase Velocities (size ∝ log10(Power))')
ax6.set_xlim(0, 25)
ax6.set_ylim(0, 6)
ax6.grid(True, alpha=0.3)

# 理論値を追加
for i, params in enumerate(wave_params):
    ax6.axhline(y=params['c'], color=f'C{i}', linestyle='--', 
                label=f"Wave {i+1}: c = {params['c']:.2f} m/s", alpha=0.8)
ax6.legend(loc='right')

plt.tight_layout()